## TODO

https://www.kaggle.com/rhtsingh/on-stability-of-few-sample-transformer-fine-tuning

https://arxiv.org/pdf/2006.05987.pdf

https://arxiv.org/pdf/2006.04884.pdf

Abstract

Тонкая настройка предварительно обученных языковых моделей на основе трансформаторов, таких как BERT, стала обычной практикой, доминирующей в списках лидеров в различных тестах NLP. Несмотря на высокие эмпирические характеристики отлаженных моделей, тонкая настройка - это нестабильный процесс: обучение одной и той же модели с несколькими случайными начальными числами может привести к большим отклонениям в производительности задачи. Предыдущая литература (Devlin et al., 2019; Lee et al., 2020; Dodge et al., 2020) идентифицировала две потенциальные причины наблюдаемой нестабильности: катастрофическое забывание и небольшой размер наборов данных для точной настройки. В этой статье мы показываем, что обе гипотезы не могут объяснить неустойчивость тонкой настройки. Мы анализируем BERT, RoBERTa и ALBERT, точно настроенные на трех часто используемых наборах данных из теста GLUE, и показываем, что наблюдаемая нестабильность вызвана трудностями оптимизации, которые приводят к исчезающим градиентам. Кроме того, мы показываем, что оставшаяся дисперсия производительности последующих задач может быть отнесена на счет различий в обобщении, когда точно настроенные модели с потерями при обучении samw демонстрируют заметно разную производительность тестов. Основываясь на нашем анализе, мы представляем простую, но надежную основу, которая делает точную настройку моделей на основе BERT значительно более стабильными, чем предложенные ранее подходы.

FINE-TUNING

we fine-tune uncased BERTLARGE (henceforth BERT) using:
- batch size of 16
- learning rate of 2e−5.

The learning rate is linearly increased from 0 to 2e−5 for the first 10% of iterations—which is known as a warmup—and linearly decreased to 0 afterward.
We apply dropout with probability p = 0.1 and weight decay with λ = 0.01. We train for 3 epochs on all datasets and use global gradient clipping. Following Devlin et al. (2019), we use the AdamW optimizer
(Loshchilov and Hutter, 2019) without bias correction

Отметим, что по сравнению с BERT и RoBERTa, и ALBERT имеют немного другие гиперпараметры. В частности, RoBERTa использует спад веса с λ = 0,1 и без
отсечение градиента, и ALBERT не использует выпадение. 


In [ ]:
from IPython.display import clear_output, Image
!pip install transformers
clear_output()

In [ ]:
import re
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch import nn, optim
import transformers

path_tr = '/content/drive/MyDrive/CommonLit/input/train.csv'
path_test = '/content/drive/MyDrive/CommonLit/input/test.csv'
path_sub = '/content/drive/MyDrive/CommonLit/input/sample_submission.csv'

SEED =13
np.random.seed(SEED)
torch.manual_seed(SEED)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = transformers.AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path='roberta-base')
model_config = transformers.AutoConfig.from_pretrained(
    pretrained_model_name_or_path='roberta-base',
    num_labels=1)
model = transformers.AutoModel.from_pretrained(
    pretrained_model_name_or_path='roberta-base',
    config=model_config)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
df = pd.read_csv(path_tr)

In [ ]:
txt = df.excerpt[:6].values
tkzr = tokenizer.batch_encode_plus(
    list(txt), # batch_text_or_text_pairs has to be a list (got <class 'numpy.ndarray'>)
    truncation=True,
    max_length=256,
    padding='max_length',
    add_special_tokens=True,
    return_attention_mask=True,
    return_token_type_ids=False,
    return_tensors='pt'
)          

In [ ]:
out = model(**tkzr)
out

BaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                               tensor([[[-0.0676,  0.0801,  0.0089,  ..., -0.0234, -0.0443, -0.0295],
                                                        [-0.0136,  0.2424, -0.1040,  ..., -0.4047,  0.0952, -0.0992],
                                                        [ 0.0357, -0.0299,  0.1018,  ..., -0.2944,  0.0626, -0.1154],
                                                        ...,
                                                        [ 0.0610, -0.0185,  0.1139,  ...,  0.2247,  0.1071,  0.0041],
                                                        [ 0.0610, -0.0185,  0.1139,  ...,  0.2247,  0.1071,  0.0041],
                                                        [ 0.0610, -0.0185,  0.1139,  ...,  0.2247,  0.1071,  0.0041]],
                                               
                                                       [[-0.0829,  0.1356,  0.0139,  ..., -0.0217, -0.0603, -0.049